# Библиотеки

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

# Функции (метрики качества)

In [2]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

# Получаем данные

In [3]:
df_train = pd.read_csv('df_train.csv', sep=';')
df_test = pd.read_csv('df_test.csv', sep=';')

In [4]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))

In [5]:
df_train.head()

,Id,Data,Target
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601..."
3,3,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569...","[6011, 6011, 6010, 6011, 6011, 4814, 4814, 601..."
4,4,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481...","[5499, 6011, 4814, 4829, 5200, 5411, 5499, 591..."


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7033 entries, 0 to 7032
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      7033 non-null   int64 
 1   Data    7033 non-null   object
 2   Target  7033 non-null   object
dtypes: int64(1), object(2)
memory usage: 165.0+ KB


In [8]:
df_test.head()

,Id,Data
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482..."
3,3,"[6011, 4722, 4722, 4722, 4814, 6011, 6011, 482..."
4,4,"[4814, 4814, 4814, 6011, 4814, 4814, 4814, 481..."


In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7033 entries, 0 to 7032
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      7033 non-null   int64 
 1   Data    7033 non-null   object
dtypes: int64(1), object(1)
memory usage: 110.0+ KB


# Baseline 1: топ10 MCC-кодов из train-части
## MCC-код и соответствующее количество вхождений в train-часть

In [ ]:
top10_codes = df_train['Data'].explode().value_counts().head(10)
top10_codes

In [ ]:
mapk(df_train['Target'], [top10_codes.index]*len(df_train))

# Baseline 2: cамые популярные транзакции пользователя.
## Если таких менее 10, то замешиваются топ10 популярных из всей выборки.

In [ ]:
def get_top_codes(transactions, top_n=10, drop_from=5):
    transactions_stats = sorted(
        Counter(transactions).items(),
        key=lambda x: x[1],
        reverse=True
    )[:top_n]


    top_codes = [mcc_code for (mcc_code, count) in transactions_stats if count >= drop_from]
    top_codes += list(top10_codes.index)

    return top_codes[:10]

In [ ]:
df_train['pred_baseline_2'] = df_train['Data'].apply(get_top_codes)
mapk(df_train['Target'], df_train['pred_baseline_2'])

# Baseline 3: категория по MCC-кодам

In [ ]:
# https://mnogo-kreditov.ru/bankovskie-karty/mcc-code.html?utm_referrer=https%3A%2F%2Fwww.google.nl%2F


# Submission

In [ ]:
df_test['Predicted'] = df_test['Data'].apply(get_top_codes)

In [ ]:
submission_baseline_2 = df_test[['Id', 'Predicted']]
submission_baseline_2['Predicted'] = submission_baseline_2.Predicted.astype(str).str.replace(',', '')
submission_baseline_2.to_csv('submission_baseline_2.csv', index=False)